In [1]:
from utils import efficientnet
blocks_args, global_params = efficientnet(width_coefficient=1.0, depth_coefficient=1.0, dropout_rate=0.7, image_size=[263, 15])

In [3]:
print(global_params)
print()
for idx, block_args in enumerate(blocks_args):
    print(f'MBConvblock {idx}: ', block_args)
    print()

GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=[263, 15], dropout_rate=0.7, num_classes=1, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=None)

MBConvblock 0:  BlockArgs(num_repeat=1, conv_kernel_size=(5, 1), pool_kernel_size=(3, 1), conv_stride=(1, 1), pool_stride=(3, 1), expand_ratio=1, input_filters=2, output_filters=4, se_ratio=0.25, id_skip=True)

MBConvblock 1:  BlockArgs(num_repeat=1, conv_kernel_size=(3, 1), pool_kernel_size=(2, 1), conv_stride=(1, 1), pool_stride=(2, 1), expand_ratio=1, input_filters=4, output_filters=8, se_ratio=0.25, id_skip=True)

MBConvblock 2:  BlockArgs(num_repeat=1, conv_kernel_size=(3, 3), pool_kernel_size=(2, 2), conv_stride=(1, 1), pool_stride=(2, 2), expand_ratio=1, input_filters=8, output_filters=16, se_ratio=0.25, id_skip=True)



In [14]:
from utils import round_filters
out_channels = round_filters(1, global_params)

In [15]:
out_channels

8

In [12]:
from utils import Conv2dStaticSamePadding, MaxPool2dStaticSamePadding
from functools import partial

bn_mom = 1 - global_params.batch_norm_momentum
bn_eps = global_params.batch_norm_epsilon
image_size = global_params.image_size
Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
MaxPool2d = partial(MaxPool2dStaticSamePadding_, image_size=image_size)

In [13]:
from torch import nn
from utils import round_filters, calculate_output_image_size, MemoryEfficientSwish

# Stem
image_size = [263, 15]
in_channels = 1
out_channels = round_filters(8, global_params)
_conv_stem = Conv2d(in_channels, out_channels, kernel_size=(5, 1), stride=(1, 1), bias=False, image_size=image_size)
_bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
_max_pool = MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
_swish = MemoryEfficientSwish()
image_size = calculate_output_image_size(image_size, stride=(1, 1))
image_size

[263, 15]

In [1]:
import kincnn
import torch

In [2]:
x = torch.randn(1, 1, 263, 15)
model = kincnn.EfficientNet.from_name('efficientnet-phospho-B-15')

In [3]:
print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    1, 8, kernel_size=(5, 1), stride=(1, 1), bias=False
    (static_padding): ZeroPad2d((0, 0, 2, 2))
  )
  (_bn0): BatchNorm2d(8, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_max_pooling): MaxPool2dStaticSamePadding(
    kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=[1, 1], ceil_mode=False
    (static_padding): ZeroPad2d((0, 0, 0, 1))
  )
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        8, 8, kernel_size=(5, 1), stride=(1, 1), groups=8, bias=False
        (static_padding): ZeroPad2d((0, 0, 2, 2))
      )
      (_bn1): BatchNorm2d(8, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_depthwise_max_pooling): MaxPool2dStaticSamePadding(
        kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=[1, 1], ceil_mode=False
        (static_padding): Identity()
      )
      (_se_reduce): Conv2dS

In [4]:
model(x)

tensor([[0.5436]], grad_fn=<SigmoidBackward0>)

In [7]:
import pytorch_model_summary

In [8]:
print(pytorch_model_summary.summary(model, torch.zeros(1, 1, 263, 15), show_input=True, show_hierarchical=False, max_depth=2, batch_size=1))

-------------------------------------------------------------------------------------
                    Layer (type)         Input Shape         Param #     Tr. Param #
                     ZeroPad2d-1     [1, 1, 263, 15]               0               0
                   BatchNorm2d-2     [1, 8, 263, 15]              16              16
                     ZeroPad2d-3     [1, 8, 263, 15]               0               0
          MemoryEfficientSwish-4     [1, 8, 132, 15]               0               0
       Conv2dStaticSamePadding-5     [1, 8, 132, 15]              40              40
                   BatchNorm2d-6     [1, 8, 132, 15]              16              16
    MaxPool2dStaticSamePadding-7     [1, 8, 132, 15]               0               0
          MemoryEfficientSwish-8      [1, 8, 44, 15]               0               0
       Conv2dStaticSamePadding-9        [1, 8, 1, 1]              18              18
      Conv2dStaticSamePadding-10        [1, 2, 1, 1]            

In [15]:
from torchsummary import summary
summary(model, input_size=(1, 1, 263, 15))

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [2, 1, 1, 267, 15]

In [20]:
from utils import round_filters
out_channels = round_filters(
            2, global_params
        )  # number of output channels

In [21]:
global_params

GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=[263, 15], dropout_rate=0.7, num_classes=1, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=None)

In [22]:
out_channels

8